A simple autocomplete model  
Given 3 characters, pick the next logical, syntactical, semantical character

In [1]:
import numpy as np
import tensorflow as tf

#### Steps
1. Generate a dictionary of character-number {chr: num}  (character pool)
2. Generate instances of length-4-words (training data)
3. Generate batches (one-hot)
4. Set hyperparameters
5. Set placeholders
6. Set variables
7. Create LSTM cell
8. Train
9. Test

In [2]:
# Step 1
char_arr = list('abcdefghijklmnopqrstuvwxyz')
num_dic = {v: i for i, v in enumerate(char_arr)}
dic_len = len(num_dic)

In [3]:
# Step 2
seq_data = ['word', 'wood', 'deep', 'dive', 'cold',
            'cool', 'load', 'love', 'kiss', 'kind',
            'deal', 'with', 'hate', 'halo', 'lone']

In [4]:
# Step 3
def make_batch(seq_data):
    input_batch, target_batch = [], []
    
    for seq in seq_data:
        inpt = [num_dic[w] for w in seq[:-1]] # Remove fourth character
        target = num_dic[seq[-1]] # Get only fourth character
        # Each input number from num_dic becomes an one-hot vector
        input_batch.append(np.eye(dic_len)[inpt]) # Create identity matrix
        target_batch.append(target)
    return input_batch, target_batch

In [5]:
make_batch(seq_data)[0][0] # First three characters in vector (input_batch)

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [6]:
make_batch(seq_data)[1] # The fourth characters (target_batch)

[3, 3, 15, 4, 3, 11, 3, 4, 18, 3, 11, 7, 4, 14, 4]

In [7]:
# Step 4
learning_rate = 0.01
n_hidden = 128 # Number of hidden states
total_epoch = 30
n_step = 3 # Three characters to read
n_input = n_class = dic_len # Number of characters

In [8]:
# Step 5
with tf.device('/gpu:0'): # Run with GPU
    X = tf.placeholder(tf.float32, [None, n_step, n_input])
    Y = tf.placeholder(tf.int32, [None])
# Running with GPU is quite useless in this case because toy dataset is very small
# and the structure is very simple.
# But it is still written because, because. (Apparently just did  it for no reason.)

In [9]:
# Step 7
W = tf.Variable(tf.random_normal([n_hidden, n_class]))
b = tf.Variable(tf.random_normal([n_class]))

In [10]:
# Step 7
cell1 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
# A cell with dropout is usually stronger against noise
cell1 = tf.nn.rnn_cell.DropoutWrapper(cell1, output_keep_prob=0.5) # Let 50% pass
# A cell without dropout is usually stronger in clean dataset
cell2 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)

In [11]:
multi_cell = tf.nn.rnn_cell.MultiRNNCell([cell1, cell2])
outputs, states = tf.nn.dynamic_rnn(multi_cell, X, dtype=tf.float32)

In [12]:
outputs = tf.transpose(outputs, [1, 0, 2])
outputs = outputs[-1]
model = tf.matmul(outputs, W) + b

In [13]:
cost = tf.reduce_mean(
    tf.nn.sparse_softmax_cross_entropy_with_logits(logits=model, labels=Y)
)

optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [14]:
# Step 8
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    input_batch, target_batch = make_batch(seq_data)

    for epoch in range(total_epoch):
        _, loss = sess.run([optimizer, cost],
                           feed_dict={X: input_batch, Y: target_batch})
        print('Epoch: {:03}, Cost = {:.6f}'.format(epoch + 1, loss))

Epoch: 001, Cost = 3.289615
Epoch: 002, Cost = 2.545197
Epoch: 003, Cost = 1.628287
Epoch: 004, Cost = 1.387787
Epoch: 005, Cost = 1.120811
Epoch: 006, Cost = 0.867861
Epoch: 007, Cost = 0.816918
Epoch: 008, Cost = 0.494083
Epoch: 009, Cost = 0.402080
Epoch: 010, Cost = 0.456980
Epoch: 011, Cost = 0.635595
Epoch: 012, Cost = 0.412365
Epoch: 013, Cost = 0.496150
Epoch: 014, Cost = 0.396356
Epoch: 015, Cost = 0.523487
Epoch: 016, Cost = 0.291665
Epoch: 017, Cost = 0.247989
Epoch: 018, Cost = 0.161046
Epoch: 019, Cost = 0.088837
Epoch: 020, Cost = 0.133633
Epoch: 021, Cost = 0.396523
Epoch: 022, Cost = 0.109906
Epoch: 023, Cost = 0.064524
Epoch: 024, Cost = 0.153590
Epoch: 025, Cost = 0.218846
Epoch: 026, Cost = 0.051808
Epoch: 027, Cost = 0.099714
Epoch: 028, Cost = 0.060895
Epoch: 029, Cost = 0.228260
Epoch: 030, Cost = 0.032484


In [15]:
# Step 9
prediction = tf.cast(tf.argmax(model, 1), tf.int32)
prediction_check = tf.equal(prediction, Y)
accuracy = tf.reduce_mean(tf.cast(prediction_check, tf.float32))

In [16]:
test_data = ['home', 'baby', 'soup', 'here']

input_batch, target_batch = make_batch(test_data)

with tf.Session() as sess:
    predict, accuracy_val = sess.run([prediction, accuracy],
                                     feed_dict={X: input_batch, Y: target_batch})
predict_words = []
for idx, val in enumerate(test_data):
    last_char = char_arr[predict[idx]]
    predict_words.append(val[:3] + last_char)
    
print('Inserted: ', [w[:3] for w in test_data])
print('Predicted: ', predict_words)
print('Accuracy: ', accuracy_val)

FailedPreconditionError: Attempting to use uninitialized value Variable
	 [[Node: Variable/read = Identity[T=DT_FLOAT, _class=["loc:@Variable"], _device="/job:localhost/replica:0/task:0/device:GPU:0"](Variable)]]
	 [[Node: Cast/_9 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_203_Cast", tensor_type=DT_INT32, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'Variable/read', defined at:
  File "/home/snu/.pyenv/versions/3.5.4/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/snu/.pyenv/versions/3.5.4/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/snu/.pyenv/versions/3.5.4/envs/deep_learning_env/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/snu/.pyenv/versi